In [ ]:
from itertools import product

import igraph
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pynauty
from scipy.linalg import ishermitian
from scipy.sparse.csgraph import connected_components
from sympy.combinatorics import Permutation, PermutationGroup

from qlinks.model.shiraishi_mori_model import SpinHalfChain, SpinOneChain
from utils import setup_igraph

np.set_printoptions(threshold=np.inf)
pd.set_option("display.max_rows", None)

## Spin-1/2 chain

In [ ]:
model = SpinHalfChain(
    n=6,
    coup_j1s=np.array([1, 1, -0.6]),
    coup_j2s=np.array([-0.8, 0, 0]),
    coup_hs=np.array([0.3, 0, 0.1]),
)
mat = model.hamiltonian.toarray()
assert ishermitian(mat, atol=1e-12)  # somewhat has rounding error
evals, evecs = np.linalg.eigh(mat)

plt.matshow(mat)
plt.colorbar()

In [ ]:
plt.plot(evals, linestyle="--", marker="o")
plt.grid()
np.where(np.abs(evals) < 1e-12)[0]

In [ ]:
i = 20
plt.plot(evecs[:, i].T, linestyle="--", marker="o")
plt.grid()
np.where(np.abs(evecs[:, i]) > 1e-6)[0]

In [ ]:
mask = np.array([11, 13, 19, 21, 22, 25, 26, 37, 38, 41, 42, 44, 50, 52])
plt.plot(model.potential_term.data[mask], linestyle="--", marker="o")
plt.grid()

In [ ]:
mat = np.zeros((2 ** model.n, 2 ** model.n))
for site in range(model.n):
    mat += model.sm_projector(site).toarray()

# mat = model.sm_projector(1)
g = nx.from_numpy_array(mat)
ig = setup_igraph(g)
igraph.plot(
    ig,
)

In [ ]:
evals, evecs = np.linalg.eigh(mat)
plt.plot(evals, linestyle="--", marker="o")
plt.grid()
np.where(np.abs(evals) < 1e-12)[0]

In [ ]:
i = 0
plt.plot(evecs[:, i].ravel(), linestyle="--", marker="o")
plt.grid()
np.where(np.abs(evecs[:, i]) > 1e-6)[0]

In [ ]:
g = nx.from_scipy_sparse_array(model.kinetic_term)
ig = igraph.Graph.from_networkx(g)

ntg = pynauty.Graph(
    ig.vcount(),
    directed=True,
    adjacency_dict=nx.to_dict_of_lists(g),
)
aut_gp = pynauty.autgrp(ntg)[0]

perm_gp = PermutationGroup([Permutation(p) for p in aut_gp])

In [ ]:
g = nx.from_scipy_sparse_array(model.kinetic_term)

highlight = [
    # np.where(np.abs(evecs[:, 231]) > 1e-6)[0],
]
# highlight = list(aut.degree_partition.values())
highlight = perm_gp.orbits()
# highlight = list(nx.bipartite.sets(g))

# highlight_color = list(mcolors.TABLEAU_COLORS.values())
# highlight_color = list(mcolors.CSS4_COLORS.values())
cmap = plt.get_cmap('Set3')
highlight_color = [mcolors.to_hex(cmap(i)) for i in range(cmap.N)]
cmap = plt.get_cmap('Set2')
highlight_color += [mcolors.to_hex(cmap(i)) for i in range(cmap.N)]
highlight_color *= 2000
# highlight_color = [
#     "dimgray",
#     "whitesmoke",
#     "deepskyblue",
#     "yellowgreen",
#     "aqua",
#     "pink",
#     "tomato",
#     "royalblue",
#     "blueviolet",
#     "cornflowerblue",
#     "limegreen",
# ]

ig = setup_igraph(g, highlight, highlight_color)

degree = np.array(list(dict(g.degree).values()))
# (bipartite, types) = ig.is_bipartite(return_types=True)
# nodes = [int(n) for n in list(sub_sub_ig.vs["label"])]
# outer_boundary = list(nx.node_boundary(g, nodes))
# sub_ig = ig.induced_subgraph(np.append(nodes, outer_boundary))

# sub_ig = ig.induced_subgraph(np.where(degree == 8)[0])
# fig, ax = plt.subplots(figsize=(6, 6), facecolor="white")
igraph.plot(
    ig,
    # layout=ig.layout_kamada_kawai(),
    # layout=ig.layout_reingold_tilford(root=[0, 25, 50, 75]),
    # layout=ig.layout_bipartite(types=types),
    vertex_size=16,
    vertex_label_size=8,
    # vertex_label_dist=1.5,
    edge_width=0.2,
    # edge_color="darkgray",
    # target="xy_graph_pbc_4.svg"
)

In [ ]:
nx.is_bipartite(g)

## Spin-1 chain

In [ ]:
model = SpinOneChain(
    n=6,
    coup_j1s=np.array([-0.8, 0.2, 0.4]),
    coup_h1s=np.array([1, 0, 0.3]),
    coup_j2s=np.array([-0.6, 0.4, 0.8]),
    coup_h2s=np.array([0, 0, -0.2]),
    coup_d=-0.4
)
mat = model.hamiltonian.toarray()
assert ishermitian(mat)
evals, evecs = np.linalg.eigh(mat)

plt.matshow(mat)
plt.colorbar()

In [ ]:
pd.DataFrame.from_dict(
    {i: j for i, j in enumerate(product([1, 0, -1], repeat=model.n))}, orient="index"
)

In [ ]:
pd.DataFrame((evecs.T @ model.q_operator @ evecs).diagonal())

In [ ]:
i = 19
plt.plot(evecs[:, i].T, linestyle="--", marker="o")
plt.grid()
np.where(np.abs(evecs[:, i]) > 1e-6)[0]

In [ ]:
g = nx.from_scipy_sparse_array(model.kinetic_term)
ig = igraph.Graph.from_networkx(g)

ntg = pynauty.Graph(
    ig.vcount(),
    directed=True,
    adjacency_dict=nx.to_dict_of_lists(g),
)
aut_gp = pynauty.autgrp(ntg)[0]

perm_gp = PermutationGroup([Permutation(p) for p in aut_gp])

In [ ]:
g = nx.from_scipy_sparse_array(model.kinetic_term)

highlight = [
    # np.where(np.abs(evecs[:, 231]) > 1e-6)[0],
]
# highlight = list(aut.degree_partition.values())
highlight = perm_gp.orbits()
# highlight = list(nx.bipartite.sets(g))

# highlight_color = list(mcolors.TABLEAU_COLORS.values())
# highlight_color = list(mcolors.CSS4_COLORS.values())
cmap = plt.get_cmap('Set3')
highlight_color = [mcolors.to_hex(cmap(i)) for i in range(cmap.N)]
cmap = plt.get_cmap('Set2')
highlight_color += [mcolors.to_hex(cmap(i)) for i in range(cmap.N)]
highlight_color *= 2000
# highlight_color = [
#     "dimgray",
#     "whitesmoke",
#     "deepskyblue",
#     "yellowgreen",
#     "aqua",
#     "pink",
#     "tomato",
#     "royalblue",
#     "blueviolet",
#     "cornflowerblue",
#     "limegreen",
# ]

ig = setup_igraph(g, highlight, highlight_color)

degree = np.array(list(dict(g.degree).values()))
# (bipartite, types) = ig.is_bipartite(return_types=True)
# nodes = [int(n) for n in list(sub_sub_ig.vs["label"])]
# outer_boundary = list(nx.node_boundary(g, nodes))
# sub_ig = ig.induced_subgraph(np.append(nodes, outer_boundary))

# sub_ig = ig.induced_subgraph(np.where(degree == 8)[0])
# fig, ax = plt.subplots(figsize=(6, 6), facecolor="white")
igraph.plot(
    ig,
    # layout=ig.layout_kamada_kawai(),
    # layout=ig.layout_reingold_tilford(root=[0, 25, 50, 75]),
    # layout=ig.layout_bipartite(types=types),
    vertex_size=16,
    vertex_label_size=8,
    # vertex_label_dist=1.5,
    edge_width=0.2,
    # edge_color="darkgray",
    # target="xy_graph_pbc_4.svg"
)

In [ ]:
sub_components = ig.connected_components(mode="weak")

for i, c in enumerate(sub_components):
    mat = nx.to_numpy_array(ig.subgraph(c).to_networkx())
    # print(repr(mat))
    print(i, mat.shape[0], mat.shape[0] - np.linalg.matrix_rank(mat), nx.is_bipartite(ig.subgraph(c).to_networkx()))